In [9]:
from rdflib import Namespace, Graph, URIRef, Literal
# Import pandas package

import pandas as pd
from difflib import SequenceMatcher

In [29]:
# change the cell width

pd.set_option("display.width", 120)

In [4]:
g_medextractor = Graph()
g_manual = Graph()

namespace_string = "http://fapranlp.de/"
namespace = Namespace(namespace_string)
# output_path = "../resources/manual_text_to_analyze.xml"
prefix = "fapra"
g_medextractor.parse("../MedExtractor/resources/evaluation_rdfs/depression.xml")
g_manual.parse("../MedExtractor/resources/manual_text_to_analyze.xml")
# print(g4)
for s, p, o in g_medextractor:
    print(s.replace(namespace_string,''),p.replace(namespace_string,''),o.replace(namespace_string+"symptom:",'').replace('_',' '))
print("----")
for s, p, o in g_manual:
    print(s.replace(namespace_string,''),p.replace(namespace_string,''),o.replace(namespace_string+"symptom:",'').replace('_',' '))
#     print((s, p, o))

depression hasSymptom changes in appetite
depression hasSymptom psychological symptoms
depression hasSymptom low self esteem
depression hasSymptom feeling irritable
depression hasSymptom tearful
depression hasSymptom no motivation
depression hasSymptom aches
depression hasSymptom pains
depression hasSymptom neglecting
depression hasSymptom lack of energy
depression hasSymptom interest
depression hasSymptom interests
depression hasSymptom loss of libido
depression hasSymptom low mood
depression hasSymptom feeling hopeless
depression hasSymptom low sex drive
depression hasSymptom feeling guilt
depression hasSymptom constipation
depression hasSymptom worried
depression hasSymptom suicidal thoughts
----
depression hasSymptom low sex drive (loss of libido)
depression hasSymptom not getting any enjoyment out of life
depression hasSymptom finding it difficult to fall asleep at night
depression hasSymptom having no motivation or interest in things
depression hasSymptom lack of energy
depressio

In [18]:
data = {'Disorder': [],
    'Relationship': [],
    'Symptom': []}
  
  
# Convert the dictionary into DataFrame
df_manual = pd.DataFrame(data)
df_medextractor = pd.DataFrame(data)


for s, p, o in g_manual:
    d2 = {"Disorder": [s.replace(namespace_string,'')], 
          "Relationship": [p.replace(namespace_string,'')],
          "Symptom": [o.replace(namespace_string+"symptom:",'').replace('_',' ')]}
    df2 = pd.DataFrame(d2)
    df_manual = pd.concat([df_manual, df2],  ignore_index=True)
    
for s, p, o in g_medextractor:
    d3 = {"Disorder": [s.replace(namespace_string,'')], 
          "Relationship": [p.replace(namespace_string,'')],
          "Symptom": [o.replace(namespace_string+"symptom:",'').replace('_',' ')]}
    df3 = pd.DataFrame(d3)
    df_medextractor = pd.concat([df_medextractor, df3],  ignore_index=True)
print(df_manual)
print("----")
print(df_medextractor)

      Disorder Relationship                                            Symptom
0   depression   hasSymptom                     low sex drive (loss of libido)
1   depression   hasSymptom              not getting any enjoyment out of life
2   depression   hasSymptom       finding it difficult to fall asleep at night
3   depression   hasSymptom         having no motivation or interest in things
4   depression   hasSymptom                                     lack of energy
5   depression   hasSymptom                                            sadness
6   depression   hasSymptom         feeling irritable and intolerant of others
7   depression   hasSymptom                                continuous low mood
8   depression   hasSymptom                       thoughts of harming yourself
9   depression   hasSymptom                         feeling anxious or worried
10  depression   hasSymptom             finding it difficult to make decisions
11  depression   hasSymptom                      cha

In [10]:

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [31]:
df_manual['Similarity'] = 0
df_manual['Symptom_Medextractor'] = 0

for index, row in df_manual.iterrows(): 
    max_similarity = 0.0
    max_symptom = ""
    
    for x in df_medextractor['Symptom']:
        similarity = similar(row['Symptom'],x)
        if similarity > max_similarity:
            max_similarity = similarity
            max_symptom = x
    if max_similarity >= 0.40: 
        df_manual.loc[index, "Similarity"] = max_similarity
        df_manual.loc[index, "Symptom_Medextractor"] = max_symptom
        # else:
        #     df_manual.loc[index, "Similarity"] = ""
        #     df_manual.loc[index, "Symptom_Medextractor"] = ""
print(df_manual[['Symptom','Symptom_Medextractor', 'Similarity']])

                                              Symptom Symptom_Medextractor  Similarity
0                      low sex drive (loss of libido)       loss of libido    0.636364
1               not getting any enjoyment out of life                    0    0.000000
2        finding it difficult to fall asleep at night                    0    0.000000
3          having no motivation or interest in things        no motivation    0.472727
4                                      lack of energy       lack of energy    1.000000
5                                             sadness                aches    0.500000
6          feeling irritable and intolerant of others    feeling irritable    0.576271
7                                 continuous low mood             low mood    0.592593
8                        thoughts of harming yourself                    0    0.000000
9                          feeling anxious or worried    feeling irritable    0.604651
10             finding it difficult to make